In [1]:
import os
from dotenv import load_dotenv, find_dotenv
result = load_dotenv(find_dotenv()) # 

#os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("_LANGCHAIN_TRACING_V2")
#os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("_LANGCHAIN_ENDPOINT")
#os.environ["LANGCHAIN_API_KEY"] = os.getenv("_LANGCHAIN_API_KEY")
#os.environ["LANGCHAIN_PROJECT"] = os.getenv("_LANGCHAIN_PROJECT")


In [2]:
from langchain_openai  import AzureChatOpenAI
from langchain.chains import ConversationChain

llm = AzureChatOpenAI(
	temperature=0,
    openai_api_version="2023-09-01-preview",
    deployment_name="GPT-4o-mini", #Deployment name
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    model_name="GPT-4o-mini"
)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)

/var/folders/5_/w60z7jfd3kggxf5884bbg2ym0000gn/T/ipykernel_48855/2324389574.py:13: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm)
/Users/gianmariaricci/develop/github/ai-playground/src/python/various/various/lib/python3.12/site-packages/pydantic/main.py:212: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [3]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [4]:
print(type(conversation.memory))

<class 'langchain.memory.buffer.ConversationBufferMemory'>


In [5]:
from langchain.callbacks import get_openai_callback
from pprint import pprint
def invoke_conversation(conversation, query):
    with get_openai_callback() as cb:
        result = conversation.invoke({"input": query})
        pprint(f'Spent a total of {cb.total_tokens} tokens with cost ${cb.total_cost:.8f}')

    return result

In [6]:
from pprint import pprint
conversation.memory.clear()
response = invoke_conversation(conversation, "Good morning AI my name is Gianmaria!")
pprint(response)

'Spent a total of 97 tokens with cost $0.00002535'
{'history': '',
 'input': 'Good morning AI my name is Gianmaria!',
 'response': "Good morning, Gianmaria! It's great to meet you! How's your day "
             'starting off? Anything exciting planned?'}


In [7]:
print(conversation.memory.buffer)

Human: Good morning AI my name is Gianmaria!
AI: Good morning, Gianmaria! It's great to meet you! How's your day starting off? Anything exciting planned?


In [8]:
response = invoke_conversation(conversation, "What is my name?")
pprint(response)

'Spent a total of 138 tokens with cost $0.00003510'
{'history': 'Human: Good morning AI my name is Gianmaria!\n'
            "AI: Good morning, Gianmaria! It's great to meet you! How's your "
            'day starting off? Anything exciting planned?',
 'input': 'What is my name?',
 'response': "Your name is Gianmaria! It's a lovely name. Do you have any "
             'special meaning behind it, or is there a story to how you got '
             'it?'}


In [9]:
print(conversation.memory.buffer)

Human: Good morning AI my name is Gianmaria!
AI: Good morning, Gianmaria! It's great to meet you! How's your day starting off? Anything exciting planned?
Human: What is my name?
AI: Your name is Gianmaria! It's a lovely name. Do you have any special meaning behind it, or is there a story to how you got it?


In [10]:
response = invoke_conversation(conversation, "What is the capital of italy?")
pprint(response)


'Spent a total of 207 tokens with cost $0.00005715'
{'history': 'Human: Good morning AI my name is Gianmaria!\n'
            "AI: Good morning, Gianmaria! It's great to meet you! How's your "
            'day starting off? Anything exciting planned?\n'
            'Human: What is my name?\n'
            "AI: Your name is Gianmaria! It's a lovely name. Do you have any "
            'special meaning behind it, or is there a story to how you got it?',
 'input': 'What is the capital of italy?',
 'response': "The capital of Italy is Rome! It's a city rich in history, known "
             'for its ancient ruins like the Colosseum and the Roman Forum, as '
             'well as its stunning architecture, delicious cuisine, and '
             'vibrant culture. Have you ever been to Rome, or is it on your '
             'travel list?'}


In [17]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

summary_buffer_memory = ConversationSummaryMemory(llm=llm, max_token_limit=40)

conversation_sum = ConversationChain(
    llm=llm,
    memory=summary_buffer_memory
)

response = invoke_conversation(conversation_sum, "Good morning AI my name is Gianmaria!")
pprint(response["response"])
print(summary_buffer_memory.buffer)

'Spent a total of 301 tokens with cost $0.00006945'
("Good morning, Gianmaria! It's great to meet you! How's your day starting "
 'off? Anything exciting planned?')
The human introduces himself as Gianmaria. The AI greets him warmly and asks how his day is starting and if he has any exciting plans.


In [18]:
print (conversation_sum.memory.buffer)

The human introduces himself as Gianmaria. The AI greets him warmly and asks how his day is starting and if he has any exciting plans.


In [20]:
response = invoke_conversation(conversation_sum, "explain hersprung russel diagram and tell me the type of our sun?")
pprint(response["response"])

'Spent a total of 1225 tokens with cost $0.00037365'
('The Hertzsprung-Russell diagram is a fascinating tool used in astronomy to '
 'classify stars based on their luminosity (brightness) and surface '
 'temperature. It essentially plots stars on a graph where the x-axis '
 'represents the surface temperature (decreasing from left to right) and the '
 'y-axis represents luminosity (increasing from bottom to top).\n'
 '\n'
 "In this diagram, you'll find several distinct regions:\n"
 '\n'
 '1. **Main Sequence**: This is where most stars, including our Sun, are '
 'located. Stars in this region fuse hydrogen into helium in their cores. The '
 'Sun is classified as a G-type main-sequence star (G dwarf), with a surface '
 'temperature of about 5,500 degrees Celsius and a luminosity of one solar '
 'unit.\n'
 '\n'
 '2. **Giants**: These stars are larger and more luminous than main-sequence '
 'stars and are found above the main sequence on the diagram.\n'
 '\n'
 '3. **Supergiants**: These ar

In [21]:
pprint(conversation_sum.memory.buffer)

('The human introduces himself as Gianmaria. The AI greets him warmly and '
 'inquires about his day and plans. Gianmaria asks the AI to explain the '
 'Hertzsprung-Russell diagram and the type of star our Sun is. The AI explains '
 'that the Hertzsprung-Russell diagram categorizes stars by luminosity and '
 'surface temperature, detailing regions such as the main sequence, giants, '
 'supergiants, and white dwarfs. It notes that the Sun is a G-type '
 'main-sequence star with a surface temperature of about 5,500 degrees Celsius '
 'and a luminosity of one solar unit, and invites Gianmaria to ask more '
 'questions if he has any.')
